### Install package

In [1]:
!pip install explicit

In [2]:
!pip install selenium

In [3]:
!pip install openpyxl

### Import

In [1]:
import requests
from bs4 import BeautifulSoup
import os.path
import itertools

from explicit import waiter, XPATH
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep

from openpyxl import Workbook, load_workbook
import unittest

In [10]:
# https://stackoverflow.com/questions/37233803/how-to-web-scrape-followers-from-instagram-web-browser
def login(driver):
    username = ""  # <username here>
    password = ""  # <password here>

    # Load page
    driver.get("https://www.instagram.com/accounts/login/")
    sleep(3)
    
    # Login
    driver.find_element_by_name("username").send_keys(username)
    driver.find_element_by_name("password").send_keys(password)
    submit = driver.find_element_by_tag_name('form')
    submit.submit()

    # Wait for the user dashboard page to load
    # WebDriverWait(driver, 15).until(
    #     EC.presence_of_element_located((By.LINK_TEXT, "See All")))
    
    sleep(3)

    driver.get("https://www.instagram.com/"+username+"/")
    

def scrape_followers(driver, account):
    # Load account page
    driver.get("https://www.instagram.com/{0}/".format(account))

    # Click the '追蹤中'/ '追蹤者' link
    sleep(2)
    driver.find_element_by_partial_link_text("追蹤者").click()  # 抓follower(追蹤者) or following(追蹤中)

    # Wait for the followers modal to load
    waiter.find_element(driver, "//div[@role='dialog']", by=XPATH)
    allfoll = int(driver.find_element_by_xpath("//li[2]/a/span").text)

    follower_css = "ul div li:nth-child({}) a.notranslate"  # Taking advange of CSS's nth-child functionality
    for group in itertools.count(start=1, step=12):
        for follower_index in range(group, group + 12):
            if follower_index > allfoll:
                raise StopIteration
            yield waiter.find_element(driver, follower_css.format(follower_index)).text
 
        last_follower = waiter.find_element(driver, follower_css.format(group+11))
        driver.execute_script("arguments[0].scrollIntoView();", last_follower)
    
        
if __name__ == "__main__":
    account = ""  # <account to check>
    driver = webdriver.Chrome()
    
    wb = Workbook()
    ws = wb.active
        
    try:
        login(driver)
        print('Followers of the "{}" account'.format(account))
        for count, follower in enumerate(scrape_followers(driver, account=account), 1):
            print("\t{}".format(follower))
            ws.append([follower])
    finally:
        driver.quit()   
        wb.save('instagran_id_'+account+'.xlsx')

Followers of the "lin1j" account
	a22177861
	mina____photo
	iverson09123
	luting_14
	hsuan_pei628
	jj124009
	leo641010
	sabrina_chung0110
	atou_51
	zero6020
	1049anna
	manchin.life
	linda870602
	tingyusandy
	mpeien
	yunyeny
	cynthia.wang__
	pin_hsiu0612
	nb__0212
	zwenru
	henry.liu_39
	jerryfaaaaaaat
	0416_jr.lyp_0726
	abc55169
	ying_hs18
	chenchenyu123
	ckcf41_jaderice
	wenweiang
	kaiyooooou
	rainbowkimberley
	sp851996811
	amy.713
	weichenlin_
	aikawackyyy
	c.yorugataningen.c
	phina522
	i_ting_915
	ivy_kuo_0125
	marktsao1206
	i_didnt_pass_iron_forging
	bao____1030
	930194puppy
	redred_19970923
	jecuteh0916
	andylee0629
	shaon_u
	shengyu0413
	luoxuanhuang
	aaas338585
	_07_09min
	w.chandler9
	wenqq1999
	ych.014
	winnie_412
	_raichuisme
	xinnnnnpan
	wingsecond
	ithrew73
	ting_tseng0405
	vivian8845
	caren89714
	yulin0111
	elisa_0828
	big___uncle
	xinying_0222
	hcl_0723
	pluslin877
	knowledge840420
	haomingwei
	jason_hsuuuu
	chendunhao
	lyxwoof_official
	behappy1117
	lo2_yaol
	168_little_j

	yanhwa__
	fsaf127
	trashko_1201
	enola0211
	booyaoting
	dennis0413
	dgk2308
	eric860929
	linyichieh
	ofunshen
	hu.zi.rei
	mandy_ku1018
	jessiewu220
	wudaniela
	jin.0303
	jane040487
	14.wen_
	putin0219


RuntimeError: generator raised StopIteration

In [4]:
def scrape_image(driver, image_id, account):
    isfamous = 0
    # Load account page
    driver.get("https://www.instagram.com/{0}/".format(account))

    sleep(2)
    
    # https://stackoverflow.com/questions/30002313/selenium-finding-elements-by-class-name-in-python
    #List<WebElement> deleteLinks = driver.find_elements(By.CLASS_NAME, "be6sR")
    
    # 所帳號的class不同
    if len(driver.find_elements(By.CLASS_NAME, "be6sR")) == 0:
        img_src = driver.find_element(By.CLASS_NAME, "_6q-tv").get_attribute("src")
    else:
        img_src = driver.find_element(By.CLASS_NAME, "be6sR").get_attribute("src")
        
    followertext = driver.find_elements(By.CLASS_NAME, "g47SY")[1].text
    
    if followertext.isnumeric() and int(followertext) == 1:
        famous = 1
    elif not followertext.isnumeric() and followertext == 'NaN':
        famous = 0
    else:
        famous = int(driver.find_elements(By.CLASS_NAME, "g47SY")[1].get_attribute("title").replace(',',''))
            
    
    print(famous)
    
    if famous > 1000:
        isfamous = 1
    
    return img_src + str(isfamous)

In [11]:
if __name__ == "__main__":
    driver = webdriver.Chrome()
    wr = load_workbook('instagran_id.xlsx')
    sheet = wr.active 
    
    wb = Workbook()
    ws = wb.active
        
    try:
        login(driver) 
        
        # https://blog.techbridge.cc/2018/10/05/how-to-use-python-manipulate-excel-spreadsheet/
        image_id = 1
        for column in sheet.columns:
            for cell in column:
                res = scrape_image(driver, image_id, account=cell.value)
                
                print(res[:-1])
                
                r = requests.get(res[:-1])
                file = open("img/" + str(image_id) +".png", "wb")
                file.write(r.content)
                file.close()
                
                ws.append([image_id, res[-1]])
                
                image_id += 1
    finally:
        driver.quit()  
        wb.save('data.xlsx')

96
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/69816866_610962046102637_8301670216887173120_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=ReBVCs6LJFcAX8h0I4M&oh=0b140b1e1124b92376383538300a6b29&oe=60690F3C
12
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/133891693_762888397638962_2736034153680461242_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=ZMjzROfIQSgAX9XV_um&oh=bca7f56629272299cec990563d600d48&oe=6069BA6A
107
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/29092695_197783800986898_7353224936942469120_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=wYTPA5ZX2-wAX84_pom&oh=c7b86c5752dd178c027f68338e155c7f&oe=60699966
92
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/37175362_651765921850614_65194262068723712_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=L5xxwiYiWEUAX-Q9_xQ&oh=2e813a1ca491cb6c3b639991130591cc&oe=606B518E
990
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.28

585
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/147409178_744845523121571_646134146247500241_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=UaKrbtrruCUAX9tmUgd&oh=78c6ff4b50f207af27085b28914311fd&oe=60697823
136
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/120505364_1068410830276308_2829640607469732800_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=5TCWn-pt2pIAX_c7G6M&oh=3f053c91611a24d3a4e3d68a0b374d8d&oe=606BBB4D
237
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/123571136_413532229665222_1049822558270934489_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=P5B9fNNu_CgAX8pVZm3&oh=0ffa19a16fa69823a1fc5eaf35dac394&oe=6069453C
420
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/120998247_759260294619235_1343117076279913276_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=nCWU11ecsmQAX9h1h25&oh=a4e89d7372cc90a94969080e4aafe065&oe=6069DEC6
103
https://instagram.ftpe8-2.fna.fbcdn.net/

317
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/47581478_384493728787333_9077681209961611264_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=rgVVv7So9lgAX8QMxhf&oh=dec0ee58c29bc3bd475a562c8d64ef7f&oe=6068A221
309
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/151731335_884411822333982_9030042636990617003_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=Jaz9zVyEocAAX9fD8ti&oh=7f0646dc9996b55b8f4ee795ce825cdb&oe=606878E0
464
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/120452874_263532368186615_2874103787440383222_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=S8nGbVuAFvwAX9KkAZg&oh=62a2bfc1a5a8686787d9221500da09f6&oe=606ADB12
489
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/131906717_114725077152761_1239513687681799156_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=oBdjaSrn1uEAX-7hSz3&oh=1b406fed05358b6dd7297256009eca9c&oe=6069F2D7
222
https://instagram.ftpe8-2.fna.fbcdn.net/v

716
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/136358502_802632793653331_6477498422428524802_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=9PLCzSQEa5kAX-Md4aj&oh=f8a473e3bb2969da5d965b192db691da&oe=606A0205
259
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/22794146_1490436251011438_2997062736455663616_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=EaDcEacJP9YAX-hYGZk&oh=95856ad0a21db2a52c06412b0f4bcd5f&oe=6069DFE1
110
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/30079464_219915908757872_7906928912402219008_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=mSNFXf_WUrMAX9SfEzC&oh=b35b9a96845cd9bd8c619f57094b19ff&oe=606A7C13
45
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s150x150/17077650_1906637932901040_172464673763885056_a.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=T_sAoQ28ox8AX9qJCyI&oh=ba5cdc6da3cf5e7d6351d22ae9b0fa15&oe=6067E27B
170
https://instagram.ftpe8-2.fna.fbcdn.net/v/t

242
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/101060404_977895045973280_4074781124646141952_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=TXHHAjXmphwAX80SFZ6&oh=077e9857932e52e6e1cdd1c9fb9acc6e&oe=60695F58
234
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/43762821_2174861692770984_286224171157946368_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=s0qMkXIRTTgAX86L7g_&oh=409a2f7ba2ce1a0ed4407660ccae38bf&oe=6068AC92
345
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/42003595_448096485597446_9009986353508646912_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=O4aPCIbR6zEAX__3kjL&oh=1d9ad8b8cbb87b68b55fa032758efe4e&oe=606AD69B
191
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/101188305_281763219535791_6375651523803742208_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=zXJAGtSetl0AX8jb-sg&oh=54bc7b85f8a31e593cb16b52458bb7ef&oe=6069AA7B
291
https://instagram.ftpe8-2.fna.fbcdn.net/v/

...

525
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/150686504_438655827380839_6325384021531731280_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=8SvQ3ez4eB4AX9xcxKz&oh=45009606894d514b7dd81e472dc88a38&oe=6068592D
455
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/73475382_441768790047946_1949401444230627328_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=YSbsV7qHPMEAX85iQse&oh=e08b900b1d495b6ce7cfa94968994253&oe=6068A513
22
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/1599520_646766992100705_849125488_a.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=l21g2NAZnxIAX_LW9bi&oh=0e96aa2010c9186c840d6be5c2be2d7b&oe=60692D7D
450
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/20184209_1493328784057598_2020373266241159168_a.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=S590Ouu4OaAAX8ZYIJb&oh=7a35d81d84d29de1a21c686187d002ba&oe=606A9D42
460
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/34324

289
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/30601903_187252515240258_2259891695279669248_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=ew40Wm9NlU4AX8Vdmw3&oh=c38282e9243fc657f516fc4725e58b11&oe=60697D45
382
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/145212575_128010119185285_366178306041308856_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=0D5ZMLYIGjoAX8B38EK&oh=e5ca60013eba4dfe69ca2ceba02a11ef&oe=6069B71B
5146
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/119217226_3337182583026392_7876137903657057864_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=NZ9Snhv1Cf8AX9EColG&oh=f7f9fb1ba79f08d5275c45792a4a430e&oe=60686AC9
234
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/67205995_410624649561382_2909925211593768960_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=0x1EkWiJVMUAX-eyKOe&oh=ab973e2a990040f4da838f3ba7561ac2&oe=606B5326
340
https://instagram.ftpe8-2.fna.fbcdn.net/v

269
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/53439419_330082194373812_4298465851805270016_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=mhiqgPJryKwAX-393vm&oh=d9965b7311c3f0e06e966da4a71cb688&oe=606BA92E
113
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/152469508_807682229818260_6665919336948514952_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=dfHW3xSGzyoAX-yJY2q&oh=624c6edb132f7434a8dbadf78ab5cafc&oe=606942A3
594
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/105964759_193216338748972_6019552086286826613_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=qi8aLwfqdxIAX8xvd7T&oh=aa01e761a6529241b7a49cf8e29965cb&oe=606BA4BC
60
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/10956620_862928350416478_2082712988_a.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=PAx52bBMkt4AX8tjYdC&oh=c8d48512ac082e46988e11f7a05b92cb&oe=606AF8A8
582
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/982

392
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/66511890_1854299901340459_305813035958665216_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=kUSQpD7m27UAX-CeQ7V&oh=22e5dde9ba1d5e4f9dab4f092fd0dd70&oe=606B787B
382
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/80703292_1965008986978879_7381504363123965952_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=FyvVYq-AcOoAX8YM31-&oh=7c5185f6f5cdd480f3945dfe156bd5e0&oe=6068C118
114
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/11208363_908089132584864_1294201753_a.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=XE9ccfr-1KAAX_Wz03Z&oh=b249ea839cb4ec81229165c21294f514&oe=6067E202
412
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/124935050_411248826709267_7086124163313167711_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=d435TKFe1nMAX_aArfA&oh=c87931b867913aab2a6bc340c4a3151d&oe=606B940F
109
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/10706779_49

787
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/69837348_1312171938943171_4041750493972660224_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=ejxbniEzrAsAX-Rl2HK&oh=289203774059a27fa599bbf2bf9ecf1f&oe=606B83F4
18
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/14334651_794459514029055_1175577354_a.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=qU_2DI85gxYAX-BN8CD&oh=2f1ede55be4fbaaaf55224da3c22d1a8&oe=6067F8D5
607
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/97145505_313039669683187_8882905100218007552_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=4a5jf5nrE9IAX9j5hFV&oh=8f6f34861815e08d54ec8d3ddcc704b2&oe=6069356D
408
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-19/s320x320/116221122_709382319639899_5132001612226062787_n.jpg?tp=1&_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_ohc=jnRx9tfvZSQAX-ZWxRO&oh=1b069c0989dc956f144ebfcca187de52&oe=606B9165
270
https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-1